## Map Mutations to Protein-Protein and Protein-Nucleic Acid Interfaces
Here we find and visualize the mutations that occur around protein-protein and protein-nucleic acid interfaces.

In [1]:
from pyspark.sql import SparkSession
from mmtfPyspark.io import mmtfReader
from mmtfPyspark.interactions import InteractionExtractor, InteractionFilter
from ipywidgets import interact, IntSlider
import py3Dmol
import pandas as pd

In [2]:
# Initialize Spark
spark = SparkSession.builder.master("local[4]").appName("3-MapToPolymerInteractions").getOrCreate()

#### Input parameters

In [3]:
distance_cutoff = 8 # distance cutoff for finding and visualizing interactions
input_file_name = 'mutations3d.csv' # mutations mapped to 3D protein structures
output_file_name = 'mutations3d_polymer_inter.csv' # mutations mapped to polymer interactions

## Read mutation file created in the previous step

In [4]:
pd.set_option('display.max_columns', None)  # show all columns
df = pd.read_csv(input_file_name)
df['pdbPosition'] = df['pdbPosition'].astype('str') # PDB residue numbers must be string to handle insertion codes
df.head()

,alignmentId,bitscore,chainId,evalue,identity,identityPositive,midlineAlign,pdbAlign,pdbFrom,pdbId,pdbNo,pdbSeg,pdbTo,refGenome,residueMapping,segStart,seqAlign,seqFrom,seqId,seqTo,updateDate,variationId,structureId,pdbPosition,pdbAminoAcid,seqIdentity,tax_id,scientific_name,structureChainId,pdbResNum,pdbSeqNum,uniprotId,uniprotNum,ID,CHROM,POS,REF,ALT,annotation,color,var_id
0,32437406,432.180,C,7.682200e-158,215.0,215.0,KPKLHYSNIRGRMESIRWLLAAAGVEFEEKFIKSAEDLDKLRNDGY...,KPKLHYSNIRGRMESIRWLLAAAGVEFEEKFIKSAEDLDKLRNDGY...,1,4acs,4acs_C_1,1,218,hgvs-grch37,"[Row(pdbAminoAcid='G', pdbPosition=83, queryAm...",4,KPKLHYSNIRGRMESIRWLLAAAGVEFEEKFIKSAEDLDKLRNDGY...,4,434735,221,2018-05-03,chr6:g.52619766C>T,4ACS,83,G,98.623853,9606,Homo sapiens,4ACS.C,83,83,P09210,83,rs147776857,6,52619766,C,T,GSTA2 missense mutation,blue,chr6:g.52619766C>T
1,32437406,432.180,C,7.682200e-158,215.0,215.0,KPKLHYSNIRGRMESIRWLLAAAGVEFEEKFIKSAEDLDKLRNDGY...,KPKLHYSNIRGRMESIRWLLAAAGVEFEEKFIKSAEDLDKLRNDGY...,1,4acs,4acs_C_1,1,218,hgvs-grch37,"[Row(pdbAminoAcid='G', pdbPosition=83, queryAm...",4,KPKLHYSNIRGRMESIRWLLAAAGVEFEEKFIKSAEDLDKLRNDGY...,4,434735,221,2018-05-03,chr6:g.52619766C>T,4ACS,83,G,98.623853,9606,Homo sapiens,4ACS.C,83,83,P09210,83,rs147776857,6,52619766,C,T,GSTA2 missense mutation,blue,chr6:g.52619766C>T
2,32437407,432.180,A,7.682200e-158,215.0,215.0,KPKLHYSNIRGRMESIRWLLAAAGVEFEEKFIKSAEDLDKLRNDGY...,KPKLHYSNIRGRMESIRWLLAAAGVEFEEKFIKSAEDLDKLRNDGY...,1,4acs,4acs_A_1,1,218,hgvs-grch37,"[Row(pdbAminoAcid='G', pdbPosition=83, queryAm...",4,KPKLHYSNIRGRMESIRWLLAAAGVEFEEKFIKSAEDLDKLRNDGY...,4,434735,221,2018-05-03,chr6:g.52619766C>T,4ACS,83,G,98.623853,9606,Homo sapiens,4ACS.A,83,83,P09210,83,rs147776857,6,52619766,C,T,GSTA2 missense mutation,blue,chr6:g.52619766C>T
3,32437407,432.180,A,7.682200e-158,215.0,215.0,KPKLHYSNIRGRMESIRWLLAAAGVEFEEKFIKSAEDLDKLRNDGY...,KPKLHYSNIRGRMESIRWLLAAAGVEFEEKFIKSAEDLDKLRNDGY...,1,4acs,4acs_A_1,1,218,hgvs-grch37,"[Row(pdbAminoAcid='G', pdbPosition=83, queryAm...",4,KPKLHYSNIRGRMESIRWLLAAAGVEFEEKFIKSAEDLDKLRNDGY...,4,434735,221,2018-05-03,chr6:g.52619766C>T,4ACS,83,G,98.623853,9606,Homo sapiens,4ACS.A,83,83,P09210,83,rs147776857,6,52619766,C,T,GSTA2 missense mutation,blue,chr6:g.52619766C>T
4,32437408,429.483,D,6.700580e-157,214.0,214.0,KPKLHYSNIRGRMESIRWLLAAAGVEFEEKFIKSAEDLDKLRNDGY...,KPKLHYSNIRGRMESIRWLLAAAGVEFEEKFIKSAEDLDKLRNDGY...,1,4acs,4acs_D_1,1,217,hgvs-grch37,"[Row(pdbAminoAcid='G', pdbPosition=83, queryAm...",4,KPKLHYSNIRGRMESIRWLLAAAGVEFEEKFIKSAEDLDKLRNDGY...,4,434735,220,2018-05-03,chr6:g.52619766C>T,4ACS,83,G,98.617512,9606,Homo sapiens,4ACS.D,83,83,P09210,83,rs147776857,6,52619766,C,T,GSTA2 missense mutation,blue,chr6:g.52619766C>T


## Create a list of unique PDB Ids

In [5]:
pdb_ids = list(df.structureId.drop_duplicates())
print("PDB Ids:", pdb_ids)

PDB Ids: ['4ACS', '2WJU', '2VCT', '2FO0', '1OPL', '5MO4', '2F4J', '2G1T', '2G2F', '2GQG', '2HIW', '3QRI', '3UE4', '2G2H', '4ZOG', '2HZI', '3QRK', '4WA9', '4TWP', '2V7A', '1DXT', '1YZI', '1YHR', '1YHE', '1YH9', '1YDZ', '1Y8W', '1Y0D', '1Y0C', '1Y0A', '1Y09', '1XZV', '1XZU', '1XZ7', '1XZ5', '1XZ4', '1XZ2', '1XYE', '1XY0', '1XXT', '4XS0', '2W6V', '1VWT', '1QXD', '3KMF', '6FQF']


## Find all polymer-polymer interactions
Find groups (residues) that interact with other polymer chains

In [6]:
structures = mmtfReader.download_mmtf_files(pdb_ids)
structures = structures.filter(lambda s: s[1].num_models == 1)  ## Currently, only structurs with 1 model are supported

queryGroupId, queryChainId, and queryGroupNumber specify the residue that interacts with another chain (targetChainId)

In [7]:
interaction_filter = InteractionFilter(distanceCutoff=distance_cutoff)

interactions = InteractionExtractor().get_polymer_interactions(structures, interaction_filter, level='group').toPandas()
interactions.head()

,structureChainId,queryGroupId,queryChainId,queryGroupNumber,targetGroupId,targetChainId,targetGroupNumber,sequenceIndex,sequence
0,4TWP.A,SER,B,501,LEU,A,387,154,DKWEMERTDITMKHKLGGGQYGEVYEGVWKKYSLTVAVKTLKEDTM...
1,4TWP.A,TYR,B,264,GLU,A,279,46,DKWEMERTDITMKHKLGGGQYGEVYEGVWKKYSLTVAVKTLKEDTM...
2,4TWP.B,THR,A,389,HIS,B,375,142,DKWEMERTDITMKHKLGGGQYGEVYEGVWKKYSLTVAVKTLKEDTM...
3,4TWP.B,THR,A,389,GLU,B,373,140,DKWEMERTDITMKHKLGGGQYGEVYEGVWKKYSLTVAVKTLKEDTM...
4,4TWP.A,LYS,B,378,LYS,A,285,52,DKWEMERTDITMKHKLGGGQYGEVYEGVWKKYSLTVAVKTLKEDTM...


## Filter mutations by joining with the interaction data

In [8]:
mt = df.merge(interactions, left_on=['structureChainId','pdbPosition'], right_on=['structureChainId','targetGroupNumber'], how='inner')
mt.head()

,alignmentId,bitscore,chainId,evalue,identity,identityPositive,midlineAlign,pdbAlign,pdbFrom,pdbId,pdbNo,pdbSeg,pdbTo,refGenome,residueMapping,segStart,seqAlign,seqFrom,seqId,seqTo,updateDate,variationId,structureId,pdbPosition,pdbAminoAcid,seqIdentity,tax_id,scientific_name,structureChainId,pdbResNum,pdbSeqNum,uniprotId,uniprotNum,ID,CHROM,POS,REF,ALT,annotation,color,var_id,queryGroupId,queryChainId,queryGroupNumber,targetGroupId,targetChainId,targetGroupNumber,sequenceIndex,sequence
0,48820809,446.432,B,4.257070e-163,220.0,221.0,AEKPKLHYSNIRGRMESIRWLLAAAGVEFEEKFIKSAEDLDKLRND...,AEKPKLHYSNIRGRMESIRWLLAAAGVEFEEKFIKSAEDLDKLRND...,1,2wju,2wju_B_1,1,221,hgvs-grch37,"[Row(pdbAminoAcid='G', pdbPosition=83, queryAm...",2,AEKPKLHYSNIRGRMESIRWLLAAAGVEFEEKFIKSAEDLDKLRND...,2,434735,222,2019-05-10,chr6:g.52619766C>T,2WJU,83,G,99.547511,9606,Homo sapiens,2WJU.B,83,83,P09210,83,rs147776857,6,52619766,C,T,GSTA2 missense mutation,blue,chr6:g.52619766C>T,HIS,C,143,GLY,B,83,82,MAEKPKLHYSNIRGRMESIRWLLAAAGVEFEEKFIKSAEDLDKLRN...
1,48820809,446.432,B,4.257070e-163,220.0,221.0,AEKPKLHYSNIRGRMESIRWLLAAAGVEFEEKFIKSAEDLDKLRND...,AEKPKLHYSNIRGRMESIRWLLAAAGVEFEEKFIKSAEDLDKLRND...,1,2wju,2wju_B_1,1,221,hgvs-grch37,"[Row(pdbAminoAcid='G', pdbPosition=83, queryAm...",2,AEKPKLHYSNIRGRMESIRWLLAAAGVEFEEKFIKSAEDLDKLRND...,2,434735,222,2019-05-10,chr6:g.52619766C>T,2WJU,83,G,99.547511,9606,Homo sapiens,2WJU.B,83,83,P09210,83,rs147776857,6,52619766,C,T,GSTA2 missense mutation,blue,chr6:g.52619766C>T,SER,C,142,GLY,B,83,82,MAEKPKLHYSNIRGRMESIRWLLAAAGVEFEEKFIKSAEDLDKLRN...
2,48820809,446.432,B,4.257070e-163,220.0,221.0,AEKPKLHYSNIRGRMESIRWLLAAAGVEFEEKFIKSAEDLDKLRND...,AEKPKLHYSNIRGRMESIRWLLAAAGVEFEEKFIKSAEDLDKLRND...,1,2wju,2wju_B_1,1,221,hgvs-grch37,"[Row(pdbAminoAcid='G', pdbPosition=83, queryAm...",2,AEKPKLHYSNIRGRMESIRWLLAAAGVEFEEKFIKSAEDLDKLRND...,2,434735,222,2019-05-10,chr6:g.52619766C>T,2WJU,83,G,99.547511,9606,Homo sapiens,2WJU.B,83,83,P09210,83,rs147776857,6,52619766,C,T,GSTA2 missense mutation,blue,chr6:g.52619766C>T,HIS,C,143,GLY,B,83,82,MAEKPKLHYSNIRGRMESIRWLLAAAGVEFEEKFIKSAEDLDKLRN...
3,48820809,446.432,B,4.257070e-163,220.0,221.0,AEKPKLHYSNIRGRMESIRWLLAAAGVEFEEKFIKSAEDLDKLRND...,AEKPKLHYSNIRGRMESIRWLLAAAGVEFEEKFIKSAEDLDKLRND...,1,2wju,2wju_B_1,1,221,hgvs-grch37,"[Row(pdbAminoAcid='G', pdbPosition=83, queryAm...",2,AEKPKLHYSNIRGRMESIRWLLAAAGVEFEEKFIKSAEDLDKLRND...,2,434735,222,2019-05-10,chr6:g.52619766C>T,2WJU,83,G,99.547511,9606,Homo sapiens,2WJU.B,83,83,P09210,83,rs147776857,6,52619766,C,T,GSTA2 missense mutation,blue,chr6:g.52619766C>T,SER,C,142,GLY,B,83,82,MAEKPKLHYSNIRGRMESIRWLLAAAGVEFEEKFIKSAEDLDKLRN...
4,48820810,446.432,A,4.257070e-163,220.0,221.0,AEKPKLHYSNIRGRMESIRWLLAAAGVEFEEKFIKSAEDLDKLRND...,AEKPKLHYSNIRGRMESIRWLLAAAGVEFEEKFIKSAEDLDKLRND...,1,2wju,2wju_A_1,1,221,hgvs-grch37,"[Row(pdbAminoAcid='G', pdbPosition=83, queryAm...",2,AEKPKLHYSNIRGRMESIRWLLAAAGVEFEEKFIKSAEDLDKLRND...,2,434735,222,2019-05-10,chr6:g.52619766C>T,2WJU,83,G,99.547511,9606,Homo sapiens,2WJU.A,83,83,P09210,83,rs147776857,6,52619766,C,T,GSTA2 missense mutation,blue,chr6:g.52619766C>T,ASP,D,47,GLY,A,83,82,MAEKPKLHYSNIRGRMESIRWLLAAAGVEFEEKFIKSAEDLDKLRN...


## Save Mappings

In [9]:
mt.to_csv(output_file_name, index=False)

In [10]:
chains = mt.groupby('structureChainId')
print("Chains:", list(chains.groups.keys()))

Chains: ['1DXT.B', '1DXT.D', '1QXD.B', '1QXD.D', '1VWT.B', '1VWT.D', '1XXT.B', '1XXT.D', '1XY0.B', '1XY0.D', '1XYE.B', '1XYE.D', '1XZ2.B', '1XZ2.D', '1XZ4.B', '1XZ4.D', '1XZ5.B', '1XZ5.D', '1XZ7.B', '1XZ7.D', '1XZU.B', '1XZU.D', '1XZV.B', '1XZV.D', '1Y09.B', '1Y09.D', '1Y0A.B', '1Y0A.D', '1Y0C.B', '1Y0C.D', '1Y0D.B', '1Y0D.D', '1Y8W.B', '1Y8W.D', '1YDZ.B', '1YDZ.D', '1YH9.B', '1YH9.D', '1YHE.B', '1YHE.D', '1YHR.B', '1YHR.D', '1YZI.B', '2G1T.A', '2G1T.B', '2G1T.C', '2G1T.D', '2G2F.A', '2VCT.A', '2VCT.B', '2W6V.B', '2W6V.D', '2WJU.A', '2WJU.B', '3KMF.C', '3KMF.G', '4XS0.B', '6FQF.C', '6FQF.D']


## View mutations grouped by protein chain

In [11]:
# Setup viewer (TODO show interactions)
def view_grouped_mutations(grouped_df, *args):
    chainIds = list(grouped_df.groups.keys())

    def view3d(show_bio_assembly, show_surface, show_labels, show_annotations, size, font, i):
        group = grouped_df.get_group(chainIds[i])
        
        pdb_id, chain_id = chainIds[i].split('.')
        global viewer
        viewer = py3Dmol.view(query='pdb:' + pdb_id, options={'doAssembly': show_bio_assembly}, width=size, height=size)

        # polymer style
        viewer.setStyle({'cartoon': {'colorscheme': 'chain', 'width': 0.6, 'opacity':0.9}})
 
        # highlight chain of interest in blue
        viewer.setStyle({'chain': chain_id},{'cartoon': {'color': 'blue'}})
        
        # non-polymer style
        viewer.setStyle({'hetflag': True}, {'stick':{'radius': 0.3, 'singleBond': False}})
    
        # don't display water molecules
        viewer.setStyle({'resn': ['HOH','DOD']}, {})
        
        rows = group.shape[0]
        for j in range(0, rows):
            res_num = str(group.iloc[j]['pdbPosition'])
            mod_res = {'resi': res_num, 'chain': chain_id} 
            col = group.iloc[j]['color']
            c_col = col + 'Carbon'
            viewer.addStyle(mod_res, {'stick':{'colorscheme':c_col, 'radius': 0.2}})
            viewer.addStyle(mod_res, {'sphere':{'color':col, 'opacity': 0.6}})  
            
            annotation = group.iloc[j]['annotation']
            mutation = group.iloc[j]['variationId']
            label = ""
            if show_labels:
                label = label + mutation + " "
            if show_annotations:
                label = label + annotation
            if show_labels or show_annotations:
                viewer.addLabel(label, {'fontSize':font,'fontColor': 'black','backgroundColor':'ivory'}, {'resi': res_num, 'chain': chain_id})
        
        viewer.zoomTo({'chain': chain_id})
        viewer.center({'chain': chain_id})
        
        if show_surface:
             viewer.addSurface(py3Dmol.SES,{'opacity':0.8,'color':'lightblue'},{'chain': chain_id})
                
        # print header
        print("PDB Id: ", pdb_id, "chain Id:", chain_id, "annotation:", annotation)
                
        return viewer.show()
       
    s_widget = IntSlider(min=0, max=len(chainIds)-1, description='Structure', continuous_update=False)
    
    return interact(view3d, show_bio_assembly=False, show_surface=False, show_labels=True, show_annotations=False, size=750, font=8, i=s_widget)

def view_image():
    return viewer.png()

In [20]:
view_grouped_mutations(chains);

interactive(children=(Checkbox(value=False, description='show_bio_assembly'), Checkbox(value=False, descriptio…

### View static image
Shown below is a static image of the 3D visualization above. To save the image, right-click on the image and choose "Copy Image" or "Save Image As..."

In [21]:
view_image()

## View one mutation at a time
Use the slider to view each mutation. Interacting residues within the `distance_cutoff` are rendered as orange sticks.

In [22]:
# Setup viewer
def view_single_mutation(df, distance_cutoff, *args):

    def view3d(show_bio_assembly, show_surface, show_labels, show_annotations, size, font, i):        
        pdb_id, chain_id = df.iloc[i]['structureChainId'].split('.')
        
        global viewer
        viewer = py3Dmol.view(query='pdb:' + pdb_id, options={'doAssembly': show_bio_assembly}, width=size, height=size)

        # polymer style
        viewer.setStyle({'cartoon': {'colorscheme': 'chain', 'width': 0.6, 'opacity':0.9}})
        
        # highlight chain of interest in blue
        viewer.setStyle({'chain': chain_id},{'cartoon': {'color': 'blue', 'opacity':0.7}})
        
        # non-polymer style
        viewer.setStyle({'hetflag': True}, {'stick':{'radius': 0.3, 'singleBond': False}})
        
        # interacting residue style
        res_num = str(df.iloc[i]['pdbPosition'])
        label = df.iloc[i]['variationId']     
        mod_res = {'resi': res_num, 'chain': chain_id} 
        col = df.iloc[i]['color']
        c_col = col + 'Carbon'
        viewer.addStyle(mod_res, {'stick':{'colorscheme':c_col, 'radius': 0.2}})
        viewer.addStyle(mod_res, {'sphere':{'color':col, 'opacity': 0.8}})   
        
        # don't display water molecules (except below for interactions)
        viewer.setStyle({'resn': ['HOH','DOD']}, {})
            
        # select residues in interacting chains by distance from mutation site (same chain is excluded)
        surroundings = {'not': {'chain': chain_id}, 'byres': True, 'within': {'distance': distance_cutoff, 'sel': mod_res}}
        
        # set style for interacting residues
        viewer.setStyle(surroundings,{'cartoon':{'color':'orange', 'width': 0.6}})
        viewer.addStyle(surroundings,{'stick':{'colorscheme':'orangeCarbon', 'radius': 0.15}})
        
        # set style for interacting waters
        waters = {'resn': ['HOH','DOD']}
        waters.update(surroundings)
        viewer.addStyle(waters,{'sphere':{'color':'orange', 'radius': 0.5}})
        
        annotation = df.iloc[i]['annotation']
        mutation = df.iloc[i]['variationId']
        label = ""
        if show_labels:
            label = label + mutation + " "
        if show_annotations:
            label = label + annotation
        if show_labels or show_annotations:
            viewer.addResLabels(surroundings, {'fontSize':font-2})
            viewer.addLabel(label, {'fontSize':font,'fontColor': 'black','backgroundColor':'ivory'}, mod_res)
            
        viewer.zoomTo(surroundings)
        viewer.center(mod_res)
        
        if show_surface:
             viewer.addSurface(py3Dmol.SES, {'opacity':0.8,'color':'lightblue'}, {'chain': chain_id})
                
        #print header
        print("PDB Id:", pdb_id, "chain Id:" , chain_id, "residue:", res_num, "mutation:", mutation, "annotation", annotation)
        
        # print any specified additional columns from the dataframe
        for a in args:
            print(a + ": " + df.iloc[i][a])
                
        return viewer.show()
       
    s_widget = IntSlider(min=0, max=len(df)-1, description='Structure', continuous_update=False)
    
    return interact(view3d, show_bio_assembly=False, show_surface=False, show_labels=True, show_annotations=False, size=750, font=12, i=s_widget)

def view_image():
    return viewer.png()

In [23]:
mt_unique = mt.drop_duplicates(["structureChainId","variationId"])

In [24]:
view_single_mutation(mt_unique, distance_cutoff);

interactive(children=(Checkbox(value=False, description='show_bio_assembly'), Checkbox(value=False, descriptio…

### View static image
Shown below is a static image of the 3D visualization above. To save the image, right-click on the image and choose "Copy Image" or "Save Image As..."

In [26]:
view_image()

In [18]:
# Shutdown Spark
spark.stop()

## Now run the next step
Map mutations occuring at protein-ligand binding sites: [4-MapToLigandInteractions.ipynb](4-MapToLigandInteractions.ipynb)